In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

mat = pd.read_csv('normMatAll.csv')
print(mat)

   8.695652173913043584e-03  ...  1.739130434782608717e-02.8942
0                  0.011494  ...                       0.011494
1                  0.000000  ...                       0.033333
2                  0.023256  ...                       0.081395
3                  0.070312  ...                       0.156250
4                  0.015625  ...                       0.039062
5                  0.007812  ...                       0.007812
6                  0.031250  ...                       0.015625

[7 rows x 49020 columns]


In [2]:
newMat = (mat.T.reset_index().T.reset_index(drop=True).set_axis([f'{i*.005}' for i in range(mat.shape[1])], axis=1)) #turn index into time stamp?
newMat = newMat.iloc[0:8,:]
print(newMat)

                        0.0  ...                        245.095
0  8.695652173913043584e-03  ...  1.739130434782608717e-02.8942
1                 0.0114943  ...                      0.0114943
2                         0  ...                      0.0333333
3                 0.0232558  ...                      0.0813953
4                 0.0703125  ...                        0.15625
5                  0.015625  ...                      0.0390625
6                 0.0078125  ...                      0.0078125
7                   0.03125  ...                       0.015625

[8 rows x 49020 columns]


In [4]:
#Plotting

In [5]:
targetMat = pd.read_csv('targetMat.csv')
newTargetMat = (targetMat.T.reset_index().T.reset_index(drop=True).set_axis([f'{i*.005}' for i in range(targetMat.shape[1])], axis=1))
newTargetMat = newTargetMat.iloc[0:7,:]
print(newTargetMat.shape)
print(newTargetMat.iloc[0,0])
print(newTargetMat)

(7, 49020)
1.000000000000000000e+00
                        0.0  ...                         245.095
0  1.000000000000000000e+00  ...  0.000000000000000000e+00.41019
1                         0  ...                               0
2                         0  ...                               0
3                         0  ...                               0
4                         0  ...                               0
5                         0  ...                               0
6                         0  ...                               1

[7 rows x 49020 columns]


In [6]:
#Target Data

targetData = np.zeros(newTargetMat.shape[1])

for i in range(newTargetMat.shape[1]):
    #print("i = ", i)
    for j in range(newTargetMat.shape[0]):
        #print("j = ", j)
        if newTargetMat.iloc[j,i] != 0:
            targetData[i] = 1+j

print(targetData)

[1. 1. 1. ... 7. 7. 7.]


In [5]:
#Training

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

X = newMat.apply(pd.to_numeric, errors='coerce')
X[np.isnan(X)] = 0
Y = pd.to_numeric(targetData)
#Y = targetData.apply(pd.to_numeric, errors='coerce')

print(X)
print(Y)

        0.0     0.005      0.01  ...   245.085    245.09   245.095
0  0.008696  0.000000  0.017391  ...  0.000000  0.000000  0.000000
1  0.011494  0.034483  0.022989  ...  0.022989  0.000000  0.011494
2  0.000000  0.000000  0.133333  ...  0.000000  0.066667  0.033333
3  0.023256  0.023256  0.058140  ...  0.023256  0.046512  0.081395
4  0.070312  0.015625  0.046875  ...  0.078125  0.023438  0.156250
5  0.015625  0.007812  0.031250  ...  0.007812  0.031250  0.039062
6  0.007812  0.023438  0.015625  ...  0.039062  0.039062  0.007812
7  0.031250  0.007812  0.015625  ...  0.039062  0.085938  0.015625

[8 rows x 49020 columns]
[1. 1. 1. ... 7. 7. 7.]


In [27]:
#windowing
sMat = np.zeros((2*X.shape[0],X.shape[1]))
                
for i in range(X.shape[1]-50):
    #print(i)
    for j in range(X.shape[0]):
        #print(j)
        sMat[j,i] = X.iloc[j,i:i+50].mean()
        sMat[2*j,i] = X.iloc[j,i:i+50].std()
        
print(sMat)

[[0.00969263 0.00966233 0.00966233 ... 0.         0.         0.        ]
 [0.02574713 0.0262069  0.0262069  ... 0.         0.         0.        ]
 [0.068      0.07       0.07133333 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.0394594  0.0395754  0.03947045 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [57]:
X = np.stack((X[:,0:sMat.shape[1]-51],sMat[:,0:sMat.shape[1]-51]))
#X = (X[:,1:sMat.shape[1]-51];sMat[:,1:sMat.shape[1]-51])
Y = Y[0:Y.shape[0]-51]
print(X)
print(X.shape)
print(Y)

AttributeError: 'list' object has no attribute 'iloc'

In [39]:
x_train, x_test, y_train, y_test = train_test_split(np.transpose(X),np.transpose(Y),test_size=.2,random_state=0)

model = MLPClassifier(hidden_layer_sizes=(100,),solver='adam',random_state = 0, max_iter=300).fit(x_train, y_train)
model.predict_proba(x_test)
model.predict(x_test)
model.score(x_test,y_test)

/var/mobile/Containers/Data/Application/C86495FF-E4D1-42D3-8C9B-BF36FD2C5961/Library/Application Support/com.rationalmatter.junoapp/python-home/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.4507861956299775

In [40]:
print(model.predict_proba(x_test))
print(model.predict(x_test))
print(model.score(x_test,y_test))

[[3.33728249e-01 2.80318037e-01 5.51263192e-02 ... 1.58916540e-01
  7.83981817e-02 5.76773498e-02]
 [2.26069554e-02 4.65826554e-01 4.68642632e-01 ... 3.11197134e-02
  9.42588304e-04 7.18582841e-03]
 [1.15118421e-01 8.61233959e-02 2.77068657e-02 ... 1.57749298e-01
  1.86484617e-01 2.41446011e-01]
 ...
 [1.54575246e-01 1.45583854e-01 8.34184237e-02 ... 2.00920467e-01
  1.20407520e-01 1.19952277e-01]
 [5.30983270e-04 5.79210794e-03 1.21787714e-02 ... 2.09322560e-03
  8.74103943e-01 1.05300968e-01]
 [5.25945845e-02 7.49651893e-03 1.01427217e-01 ... 2.24254997e-01
  5.16011474e-02 4.57520628e-01]]
[1. 3. 7. ... 5. 6. 7.]
0.4507861956299775


In [24]:
a = [1,2,3,4,5,6,7,8,9]
print(a[:5])

[1, 2, 3, 4, 5]
